In [1]:
import json
from joblib import dump, load
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.ensemble import RandomForestClassifier

import repath.data.datasets.bloodmucus as bloodm
from repath.preprocess.patching import GridPatchFinder, SlidesIndex, CombinedIndex
from repath.preprocess.tissue_detection import TissueDetectorGreyScale, SizedClosingTransform, FillHolesTransform
from repath.preprocess.tissue_detection.blood_mucus import get_slides_annots, apply_tissue_detection, get_features_list, fit_segmenter_multi, predict_segmenter, get_features, pool_blood_mucus, get_annot_areas, calc_confusion_mat_2class, calc_confusion_mat_3class, save_confusion_mat, flatten_mask_annots_list
from repath.utils.convert import get_concat_h, get_concat_v
from repath.utils.paths import project_root
from repath.utils.seeds import set_seed

from sklearn.utils.validation import check_is_fitted

"""
Global stuff
"""
experiment_name = "bloodmucus_lev4_rf"
experiment_root = project_root() / "experiments" / experiment_name

global_seed = 123

In [2]:
set_seed(global_seed)
level_label = 4

# read in slides and annotations for training
dset = bloodm.training()
len(dset)

60

In [3]:
thumbz, annotz = get_slides_annots(dset, level_label)


In [4]:
# apply tissue detection
morphology_transform1 = SizedClosingTransform(level_in=level_label)
morphology_transform2 = FillHolesTransform(level_in=level_label)
morphology_transforms = [morphology_transform1, morphology_transform2]
tissue_detector = TissueDetectorGreyScale(grey_level=0.85, morph_transform = morphology_transforms)
filtered_thumbz = apply_tissue_detection(thumbz, tissue_detector)


In [5]:
annotz_masked = flatten_mask_annots_list(annotz, filtered_thumbz)

In [6]:
from functools import partial
import itertools

from repath.preprocess.tissue_detection.multiscale_features import _mutiscale_basic_features_singlechannel


In [7]:
def multiscale_basic_features(
    image,
    multichannel=False,
    intensity=True,
    edges=True,
    texture=True,
    sigma_min=0.5,
    sigma_max=16,
    num_sigma=None,
    num_workers=None,
):
    """Local features for a single- or multi-channel nd image.
    Intensity, gradient intensity and local structure are computed at
    different scales thanks to Gaussian blurring.
    Parameters
    ----------
    image : ndarray
        Input image, which can be grayscale or multichannel.
    multichannel : bool, default False
        True if the last dimension corresponds to color channels.
    intensity : bool, default True
        If True, pixel intensities averaged over the different scales
        are added to the feature set.
    edges : bool, default True
        If True, intensities of local gradients averaged over the different
        scales are added to the feature set.
    texture : bool, default True
        If True, eigenvalues of the Hessian matrix after Gaussian blurring
        at different scales are added to the feature set.
    sigma_min : float, optional
        Smallest value of the Gaussian kernel used to average local
        neighbourhoods before extracting features.
    sigma_max : float, optional
        Largest value of the Gaussian kernel used to average local
        neighbourhoods before extracting features.
    num_sigma : int, optional
        Number of values of the Gaussian kernel between sigma_min and sigma_max.
        If None, sigma_min multiplied by powers of 2 are used.
    num_workers : int or None, optional
        The number of parallel threads to use. If set to ``None``, the full
        set of available cores are used.
    Returns
    -------
    features : np.ndarray
        Array of shape ``image.shape + (n_features,)``
    """
    
    # create tissue mask
    rr = image[:, :, 0] == 255
    gg = image[:, :, 0] == 255
    bb = image[:, :, 0] == 255
    mask = np.logical_not(np.logical_and(np.logical_and(rr, gg), bb))
    #mask = mask.flatten()
    print(mask.shape, np.sum(mask))

    if not any([intensity, edges, texture]):
        raise ValueError(
                "At least one of ``intensity``, ``edges`` or ``textures``"
                "must be True for features to be computed."
                )
    if image.ndim < 3:
        multichannel = False
    if not multichannel:
        image = image[..., np.newaxis]
    all_results = (
        _mutiscale_basic_features_singlechannel(
            image[..., dim],
            intensity=intensity,
            edges=edges,
            texture=texture,
            sigma_min=sigma_min,
            sigma_max=sigma_max,
            num_sigma=num_sigma,
            num_workers=num_workers,
        )
        for dim in range(image.shape[-1])
    )
    features = list(itertools.chain.from_iterable(all_results))
    out = np.stack(features, axis=-1)
    print(out.shape)
    out = out[mask]
    print(out.shape)
    return out

In [8]:
def get_features(filtered_thumb, sigma_min = 1, sigma_max=16, edges=False):
    features_func = partial(multiscale_basic_features,
                            intensity=True, edges=edges, texture=True,
                            sigma_min=sigma_min, sigma_max=sigma_max,
                            multichannel=True)
    features = features_func(filtered_thumb)
    return features

In [9]:
def get_features_list(filtered_thumbz, sigma_min = 1, sigma_max=16, edges=False):

    featz = []
    for idx, tt in enumerate(filtered_thumbz):
        print(idx)
        features = get_features(tt, sigma_min, sigma_max, edges)
        featz.append(features)

    return featz

In [10]:
featz = get_features_list(filtered_thumbz, edges=True)

0
(5054, 9438) 5456908
(5054, 9438, 60)
(5456908, 60)
1
(5262, 6350) 4984719
(5262, 6350, 60)
(4984719, 60)
2
(4622, 5150) 5747818
(4622, 5150, 60)
(5747818, 60)
3
(2022, 1574) 183715
(2022, 1574, 60)
(183715, 60)
4
(5006, 5598) 7289375
(5006, 5598, 60)
(7289375, 60)
5
(6158, 7758) 24186670
(6158, 7758, 60)
(24186670, 60)
6
(5342, 8670) 16517849
(5342, 8670, 60)
(16517849, 60)
7
(4878, 4622) 2778122
(4878, 4622, 60)
(2778122, 60)
8
(5966, 6750) 11159236
(5966, 6750, 60)
(11159236, 60)
9
(3022, 2702) 2433894
(3022, 2702, 60)
(2433894, 60)
10
(5726, 6286) 7422023
(5726, 6286, 60)
(7422023, 60)
11
(3878, 3470) 2675824
(3878, 3470, 60)
(2675824, 60)
12
(3214, 2574) 1818468
(3214, 2574, 60)
(1818468, 60)
13
(6142, 8222) 22196962
(6142, 8222, 60)
(22196962, 60)
14
(3934, 5646) 2943838
(3934, 5646, 60)
(2943838, 60)
15
(5838, 5966) 11467175
(5838, 5966, 60)
(11467175, 60)
16
(3470, 6670) 2082307
(3470, 6670, 60)
(2082307, 60)
17
(4070, 4070) 3487108
(4070, 4070, 60)
(3487108, 60)
18
(6030, 75

In [11]:
def fit_segmenter_multi(labels_list, features_list, clf):
    """Segmentation using labeled parts of the image and a classifier.
    Parameters
    ----------
    labels : ndarray of ints
        Image of labels. Labels >= 1 correspond to the training set and
        label 0 to unlabeled pixels to be segmented.
    features : ndarray
        Array of features, with the first dimension corresponding to the number
        of features, and the other dimensions correspond to ``labels.shape``.
    clf : classifier object
        classifier object, exposing a ``fit`` and a ``predict`` method as in
        scikit-learn's API, for example an instance of
        ``RandomForestClassifier`` or ``LogisticRegression`` classifier.
    Returns
    -------
    clf : classifier object
        classifier trained on ``labels``
    Raises
    ------
    NotFittedError if ``self.clf`` has not been fitted yet (use ``self.fit``).
    """
    training_data_all = []
    training_labels_all = []
    for idx, labels in enumerate(labels_list):
        # mask = labels > 0
        # training_data = features_list[idx][mask]
        training_data = features_list[idx]
        if idx == 0:
            training_data_all = training_data
        else:
            training_data_all = np.vstack((training_data_all, training_data))
        # training_labels = labels[mask].ravel()
        print(labels.shape)
        training_labels_all = np.hstack((training_labels_all, labels))
    print(training_data_all.shape, training_labels_all.shape)
    clf.fit(training_data_all, training_labels_all)
    print(clf, check_is_fitted(clf))
    return clf

In [12]:
def flatten_mask_annots(annots, image):
    print(np.max(image))
    rr = image[:, :, 0] == 255
    gg = image[:, :, 0] == 255
    bb = image[:, :, 0] == 255
    mask = np.logical_not(np.logical_and(np.logical_and(rr, gg), bb))
    print(mask.shape, annots.shape, np.sum(mask))
    annots = annots[mask]
    print(annots.shape)
    return annots

def flatten_mask_annots_list(annotz, thumbz):
    annots_out = []
    for img, ann in zip(thumbz, annotz):
        annot = flatten_mask_annots(ann, img)
        annots_out.append(annot)
    return annots_out

In [13]:
flatten_mask_annots(annotz[0], filtered_thumbz[0])

255
(5054, 9438) (5054, 9438) 5456908
(5456908,)


array([0, 0, 0, ..., 0, 0, 0])

In [14]:
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=10, max_samples=0.25)
clf = fit_segmenter_multi(annotz_masked, featz, clf)

(5456908,)
(4984719,)
(5747818,)
(183715,)
(7289375,)
(24186670,)
(16517849,)
(2778122,)
(11159236,)
(2433894,)
(7422023,)
(2675824,)
(1818468,)
(22196962,)
(2943838,)
(11467175,)
(2082307,)
(3487108,)
(24099136,)
(2396909,)
(8347799,)
(10069743,)
(7859905,)
(7595483,)
(823772,)
(408764,)
(4083609,)
(5655268,)
(8250768,)
(6660891,)
(10218027,)
(3370070,)
(10565996,)


MemoryError: Unable to allocate 113. GiB for an array with shape (252315124, 60) and data type float64